### Importação dos dados
Utilização da biblioteca pandas, beautiful soup e requests para obter os dados

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# url = 'https://idealsoftwares.com.br/indices/dolar2024.html'
# resp = requests.get(url)
# soup = BeautifulSoup(resp.text, 'html5lib')

d_cotacoes = []

In [ ]:
 def getCotacaoUS(*urls):
    for url in urls:
        #url = 'https://idealsoftwares.com.br/indices/dolar2024.html'
        #url = 'https://idealsoftwares.com.br/indices/dolar2023.html'
        resp = requests.get(url)

        soup = BeautifulSoup(resp.text, 'html5lib')
        #Retorna todos os tbody's
        tbodies = soup.find_all('tbody')

        for tbody in tbodies:
            #Retorna todas as linhas
            rows = tbody.find_all('tr')
            for row in rows:
                #Retorna as informaçõe somente onde houver 03 colunas (data, compra e venda)
                columns = row.find_all('td')
                if len(columns) == 3:
                    date = columns[0].get_text(strip=True)
                    compra = columns[1].get_text(strip=True)
                    venda = columns[2].get_text(strip=True)
                #Adicionando ao dicionário
                    d_cotacoes.append({'Data':date,'Moeda':'US', 'Compra':compra, 'Venda':venda})

In [ ]:
#Importação das cotações dos respectivos períodos informados
getCotacaoUS('https://idealsoftwares.com.br/indices/dolar2024.html','https://idealsoftwares.com.br/indices/dolar2023.html','https://idealsoftwares.com.br/indices/dolar2022.html','https://idealsoftwares.com.br/indices/dolar2021.html','https://idealsoftwares.com.br/indices/dolar2020.html')


In [ ]:
#d_cotacoes[1]

In [ ]:
#Converte lista para um dataframe
d_cotacoes = pd.DataFrame(d_cotacoes) 

In [ ]:
d_cotacoes.head()
#d_cotacoes.describe()

In [ ]:
d_cotacoes.dtypes

In [ ]:
#d_cotacoes['Data'].astype(date)

# Convertendo a coluna 'Data' para o tipo datetime
d_cotacoes['Data'] = pd.to_datetime(d_cotacoes['Data'], dayfirst=True)
d_cotacoes['Compra'] = pd.to_numeric(d_cotacoes['Compra'].str.replace(',','.'), downcast='float')
d_cotacoes['Venda'] = pd.to_numeric(d_cotacoes['Venda'].str.replace(',','.'), downcast='float')

#Removendo registros que estiverem sem dados NaN
d_cotacoes = d_cotacoes.dropna()


In [ ]:
#d_cotacoes['Venda'] = pd.to_numeric(d_cotacoes['Compra'].str.replace(',','.'), downcast='float')
d_cotacoes.dtypes

In [ ]:
d_cotacoes.describe()

### Conexão com banco de dados
Nesta seção será realizada a conexão com o banco de dados, para após tratamento dos dados, realizar a inserção das informações na tabela

In [ ]:
import psycopg2
from psycopg2 import Error

In [ ]:
try:
    # Parâmetros da conexão
    conn_params = {
        "host": "localhost",
        "database": "pd",
        "user": "postgres",
        "password": "postgres"
    }

    # Conectar ao PostgreSQL
    conn = psycopg2.connect(**conn_params)

    # Criar um cursor
    cursor = conn.cursor()

    # Executar uma consulta SQL
    cursor.execute("SELECT * FROM ms.ms_vlcotacao;")
    
    # Recuperar os resultados da consulta
    rows = cursor.fetchall()
    
    # Exibir os resultados
    for row in rows:
        print(row)

except Error as e:
    print(f"Erro ao conectar ou manipular o PostgreSQL: {e}")
 

In [ ]:
#Loop pelos registros para inserir no banco de dados
for index, row in d_cotacoes.iterrows():
    data = row['Data']
    moeda = row['Moeda']
    compra = row['Compra']
    venda = row['Venda']
    
    # Comando SQL de INSERT
    sql = f"INSERT INTO ms.ms_vlcotacao (\"Data\",moeda,compra,venda) VALUES ('{data}','{moeda}',{compra},{venda})"
    
    # Executar o comando SQL
    cursor.execute(sql)
    
# Confirmar a transação
conn.commit()